<a href="https://colab.research.google.com/github/mechaturtles/SOST4Website/blob/master/Functions_without_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Estimated Glucose Values (egvs) dictionary - used for Functions 1 & 2. Deliverd from the Dexcom G6 glucose monitor

    Keys:
      time = ["hours(24hr)minutes(base 100)", ...]
      realtimeValue1 = [egvs data as floats]
      trend1 = [egvs trendrate data as floats]

      Events dictionary list - t_(time), there is an event for each time point delivered (if no event occurs then the event is None)
        Event types are stored in first position of array (carbs, Exercise, insulin, None)
          [Carbs, grams]
          [Exercise, type (light, medium, hard), hours (float)]
          [insulin, type (fastActing, longActing), units (float)]

In [2]:
egvs = dict(
    #Hour (24 hr), minute (base 100), strings
    time = [ "1600", "1650", "1700", "1750", "1800", "1850", "1900", "1950", "2000", "2050"],
    #Estimated Glucose Value data, test case #1 
    realtimeValue1 = [130.0, 120.0, 110.0, 90.0, 80.0, 70.0, 60.0, 50.0, 40.0, 30.0],
    #an 'event' is stored at each time, if no event occurs then the value is null. Event types are carbs, insulin, and Exercise
    Events = dict(
          t_1600 = ["carbs", 35],
          t_1650 = ["Exercise", "light", 2.0],
          t_1700 = ["insulin", "fastActing", 2.2],
          t_1750 = [None],
          t_1800 = ["Exercise", "medium", 1.0],
          t_1850 = ["insulin", "fastActing", 1.8],
          t_1900 = ["Exercise", "Hard", 0.5],
          t_1950 = ["insulin", "longActing", 17],
          t_2000 = ["carbs", 50],
          t_2050 = ["Exercise", "light", 0.5], 
        ),
    #trend is the rate at which EGV is changing at any one data point
    trend = dict(
        flat1 = [ -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0 ],
    ))

DangerZone: Alerts time spend in a hyper/hypo glycemic state.

Future plan: instead of alerting (as if reporting live) will be changed to store the number of instances in hypo/hyper state, the average EGV during that state, and the estimated amount of time in the state at the end of a "data reporting period"

In [ ]:
def DangerZones(TestCase, Benchmark):
    TC = int(TestCase, 10)
    storeHigh = [0] * 5
    storeLow = [0] * 5
    for i in range(len( egvs["realtimeValue" + TestCase] ) ): #iterates for total number egvs datapoints
        null = 1 
        Value = egvs["realtimeValue" + TestCase][i]
        trend = egvs["trend"]["flat" + TestCase][i]
        if  trend > 0.0: #Where is the egvs going
          Direction = "Hyperglycemia"
        elif trend < 0.0:
          Direction = "Hypoglycemia"

        if (Value >= 130) or (Value <= 70): #hyper/hypo glycemic state
          null = 0 
        
        if null == 0: #Records the number of instances that a hypo/hyper glycemic state is entered
          if Value > 130:
             storeHigh[TC] = storeHigh[TC] + 1
          elif Value < 70:
            storeLow[TC] = storeLow[TC] + 1
            
          if PatientBenchmark <= (storeLow[TC] or storeHigh[TC]): #alerts if over benchmark
            print("A L E R T - Dangerous trend:", Direction, ",", Value, "\n")

        null = 1
        Direction = None

    return

TestCase = "1" #input("Please input test case (# 1-4):")
PatientBenchmark = 3 #int(input("Please enter the patient's benchmark for alerts:"))

DangerZones(TestCase, PatientBenchmark)

Function for observing the number of instances that carbs are consumed and the sum that is consumed in the study period

In [10]:
def CarbReport(): #uses egvs dictionary to report the sum of Carbs administered in the study period.
  CarbSum = 0.0
  for i in egvs["time"]:       
      if egvs["Events"]["t_" + i][0] == "carbs":
        CarbSum = CarbSum + egvs["Events"]["t_" + i][1]

  Result = time = "From %s:%02.0f to %s:%02.0f, %02.0f g of carbohydrates were consumed." % (egvs["time"][0][:2], (int(egvs["time"][0][2:4], 10) * 60 / 100), egvs["time"][-1][:2], int(egvs["time"][-1][2:4], 10) * 60 / 100, CarbSum)
  return Result
        
Result_CarbReport = CarbReport()

From 16:00 to 20:30, 85 g of carbohydrates were consumed.
